### 1章 セレクションバイアスとRCT

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import random

In [2]:
pd.set_option('max_columns', 500)
pd.set_option('max_rows', 500)

from IPython.display import display

In [3]:
ulr = "http://www.minethatdata.com/Kevin_Hillstrom_MineThatData_E-MailAnalytics_DataMiningChallenge_2008.03.20.csv"

In [4]:
data = pd.read_csv(ulr)

### dataの各カラムの内容は下記サイト参照
https://blog.minethatdata.com/2008/03/minethatdata-e-mail-analytics-and-data.html

### データの概要

In [5]:
display(data.head(3))
display(data.tail(3))
display(data.shape)
display(data.dtypes)

,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend
0,10,2) $100 - $200,142.44,1,0,Surburban,0,Phone,Womens E-Mail,0,0,0.0
1,6,3) $200 - $350,329.08,1,1,Rural,1,Web,No E-Mail,0,0,0.0
2,7,2) $100 - $200,180.65,0,1,Surburban,1,Web,Womens E-Mail,0,0,0.0


,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend
63997,6,1) $0 - $100,29.99,1,0,Urban,1,Phone,Mens E-Mail,0,0,0.0
63998,1,5) $500 - $750,552.94,1,0,Surburban,1,Multichannel,Womens E-Mail,0,0,0.0
63999,1,4) $350 - $500,472.82,0,1,Surburban,0,Web,Mens E-Mail,0,0,0.0


(64000, 12)

recency              int64
history_segment     object
history            float64
mens                 int64
womens               int64
zip_code            object
newbie               int64
channel             object
segment             object
visit                int64
conversion           int64
spend              float64
dtype: object

In [6]:
display(data.describe(include='all'))

,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend
count,64000.000000,64000,64000.000000,64000.000000,64000.000000,64000,64000.000000,64000,64000,64000.000000,64000.000000,64000.000000
unique,NaN,7,NaN,NaN,NaN,3,NaN,3,3,NaN,NaN,NaN
top,NaN,1) $0 - $100,NaN,NaN,NaN,Surburban,NaN,Web,Womens E-Mail,NaN,NaN,NaN
freq,NaN,22970,NaN,NaN,NaN,28776,NaN,28217,21387,NaN,NaN,NaN
mean,5.763734,NaN,242.085656,0.551031,0.549719,NaN,0.502250,NaN,NaN,0.146781,0.009031,1.050908
std,3.507592,NaN,256.158608,0.497393,0.497526,NaN,0.499999,NaN,NaN,0.353890,0.094604,15.036448
min,1.000000,NaN,29.990000,0.000000,0.000000,NaN,0.000000,NaN,NaN,0.000000,0.000000,0.000000
25%,2.000000,NaN,64.660000,0.000000,0.000000,NaN,0.000000,NaN,NaN,0.000000,0.000000,0.000000
50%,6.000000,NaN,158.110000,1.000000,1.000000,NaN,1.000000,NaN,NaN,0.000000,0.000000,0.000000
75%,9.000000,NaN,325.657500,1.000000,1.000000,NaN,1.000000,NaN,NaN,0.000000,0.000000,0.000000


In [7]:
for i in data.columns:
    print(f"{i}: {np.sort(data[i].unique())}\n")

recency: [ 1  2  3  4  5  6  7  8  9 10 11 12]

history_segment: ['1) $0 - $100' '2) $100 - $200' '3) $200 - $350' '4) $350 - $500'
 '5) $500 - $750' '6) $750 - $1,000' '7) $1,000 +']

history: [  29.99   30.     30.01 ... 3040.2  3215.97 3345.93]

mens: [0 1]

womens: [0 1]

zip_code: ['Rural' 'Surburban' 'Urban']

newbie: [0 1]

channel: ['Multichannel' 'Phone' 'Web']

segment: ['Mens E-Mail' 'No E-Mail' 'Womens E-Mail']

visit: [0 1]

conversion: [0 1]

spend: [  0.    29.99  30.24  30.7   30.94  30.98  31.03  32.07  32.17  32.21
  32.22  32.43  32.51  32.99  33.02  33.13  33.32  33.45  33.51  33.66
  33.68  34.02  34.05  34.1   34.38  34.6   34.66  35.2   35.51  36.07
  36.09  36.16  36.18  36.2   36.71  36.82  36.89  37.27  37.42  37.65
  37.79  37.87  38.12  38.23  38.85  39.81  39.91  40.43  41.18  41.38
  41.45  42.66  43.49  43.7   43.84  43.9   43.96  44.12  44.14  44.26
  44.64  44.7   45.18  45.83  46.38  46.77  47.36  47.39  47.53  47.7
  47.87  47.98  49.21  49.55  49.7  

In [8]:
for i in data.columns:
    print(f"{i}\n{data[i].value_counts(sort=False)}\n")

recency
1     8952
2     7537
3     5904
4     5077
5     4510
6     4605
7     4078
8     3495
9     6441
10    7565
11    3504
12    2332
Name: recency, dtype: int64

history_segment
2) $100 - $200      14254
1) $0 - $100        22970
5) $500 - $750       4911
7) $1,000 +          1308
4) $350 - $500       6409
3) $200 - $350      12289
6) $750 - $1,000     1859
Name: history_segment, dtype: int64

history
104.00    1
128.00    2
48.00     3
64.00     2
253.59    1
         ..
350.81    1
506.82    1
88.67     3
381.09    1
240.04    2
Name: history, Length: 34833, dtype: int64

mens
0    28734
1    35266
Name: mens, dtype: int64

womens
0    28818
1    35182
Name: womens, dtype: int64

zip_code
Rural         9563
Urban        25661
Surburban    28776
Name: zip_code, dtype: int64

newbie
0    31856
1    32144
Name: newbie, dtype: int64

channel
Web             28217
Phone           28021
Multichannel     7762
Name: channel, dtype: int64

segment
Mens E-Mail      21307
No E-Mail      

### データの準備

In [9]:
# 女性向けメールを除いたデータセットを作成
male_data = data[data["segment"] != "Womens E-Mail"]
male_data["segment"].unique()

array(['No E-Mail', 'Mens E-Mail'], dtype=object)

In [10]:
# 介入の有無(Mens E-Mail なら 1、そうでなければ 0)を表す treatment 変数を追加
male_data["treatment"] = 0
male_data["treatment"].mask(male_data["segment"] == "Mens E-Mail", 1, inplace=True)
male_data.head(5)

C:\Users\kojis\anaconda3\envs\Pip ML\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\kojis\anaconda3\envs\Pip ML\lib\site-packages\pandas\core\generic.py:8993: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cond, other, inplace, axis, level, errors=errors, try_cast=try_cast


,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend,treatment
1,6,3) $200 - $350,329.08,1,1,Rural,1,Web,No E-Mail,0,0,0.0,0
3,9,5) $500 - $750,675.83,1,0,Rural,1,Web,Mens E-Mail,0,0,0.0,1
8,9,5) $500 - $750,675.07,1,1,Rural,1,Phone,Mens E-Mail,0,0,0.0,1
13,2,2) $100 - $200,101.64,0,1,Urban,0,Web,Mens E-Mail,1,0,0.0,1
14,4,3) $200 - $350,241.42,0,1,Rural,1,Multichannel,No E-Mail,0,0,0.0,0


### RCTデータの集計と有意差検定

### 集計

In [11]:
# メールが配信されたグループとされなかったグループでの購入の発生確率(conversion)と購入額(spend)の平均を計算
male_data.groupby("treatment").mean()[["conversion", "spend"]]

,conversion,spend
treatment,,
0,0.005726,0.652789
1,0.012531,1.422617


In [39]:
# メールが配信されたグループとされなかったグループのカウント数
male_data["treatment"].value_counts(sort=False)

0    21306
1    21307
Name: treatment, dtype: int64

### 有意差検定

In [15]:
# 男性向けメールが配信されたグループの購買データ
mens_mail = male_data[male_data["treatment"] == 1]["spend"]

In [16]:
# メールが配信されなかったグループの購買データ
no_mail = male_data[male_data["treatment"] == 0]["spend"]

In [17]:
# 2群間でのt検定　p-valueは1.163e-07 と小さいので有意
stats.ttest_ind(mens_mail, no_mail, equal_var=True)

Ttest_indResult(statistic=5.300090294465472, pvalue=1.163200872605869e-07)

### バイアスのあるデータによる効果の検証

### バイアスのあるデータの作成

In [18]:
# 去年の購入額(history)が一定以下のデータをランダムに50%削除し、バイアスを作る

# メールが配信されていないグループでは、下記3つの条件のどれかに該当するデータをランダムに半分選んで削除する
#「去年の購入額であるhistoryが300より小さい場合」、
#「最後の購入であるrecencyが3より小さい場合」
#「接触チャネル channelが複数あることを表すMultichannelである場合」

# メールが配信されているグループでは、先ほどの3つの条件に該当しないデータをランダムに半分選んで削除する

# すなわち、購買意欲の高いユーザーにメールが多く配信されたデータを作成

In [19]:
biased_data = male_data.copy()

In [20]:
# メールが配信されておらず、かつ、3つの条件のいずれかを満たす（＝購買意欲が高い）データ
# これを半分削除することで、メールが配信されていないグループには購買意欲が低い人たちが残る
fulfill_cond_data = male_data[((male_data["treatment"] == 0) & ((male_data["history"] > 300) | (male_data["recency"] < 6) | (biased_data["channel"] == "Multichannel")))]
fulfill_cond_data.shape

(13099, 13)

In [21]:
# メールが配信されており、かつ、3つの条件のいずれも満たさない(=購買意欲が低い)データ
# これを半分削除することで、メールが配信されているグループには購買意欲が高い人が値が残る
non_fulfill_cond_data = male_data[((biased_data["treatment"] == 1) & (male_data["history"] <= 300) & (male_data["recency"] >= 6) & (male_data["channel"] != "Multichannel"))]
non_fulfill_cond_data.shape

(8277, 13)

In [22]:
# 条件を満たすデータから半分抽出する
fulfill_cond_data.sample(frac=0.5, random_state=42)

,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend,treatment
21259,1,1) $0 - $100,81.55,1,0,Surburban,0,Phone,No E-Mail,0,0,0.00,0
37881,3,2) $100 - $200,187.30,1,0,Urban,0,Web,No E-Mail,1,1,75.41,0
17308,4,1) $0 - $100,31.65,0,1,Surburban,1,Phone,No E-Mail,0,0,0.00,0
6184,2,2) $100 - $200,133.37,1,0,Surburban,1,Phone,No E-Mail,0,0,0.00,0
3546,1,5) $500 - $750,516.70,0,1,Surburban,1,Phone,No E-Mail,0,0,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25295,4,3) $200 - $350,249.87,1,1,Urban,0,Web,No E-Mail,0,0,0.00,0
42959,4,1) $0 - $100,89.11,0,1,Surburban,0,Web,No E-Mail,0,0,0.00,0
17485,2,2) $100 - $200,125.28,1,0,Urban,0,Phone,No E-Mail,0,0,0.00,0
22410,5,3) $200 - $350,224.88,1,0,Urban,0,Multichannel,No E-Mail,0,0,0.00,0


In [23]:
# 条件を満たすデータの半分を削除する
biased_data.drop(fulfill_cond_data.sample(frac=0.5, random_state=42).index, inplace=True)

In [24]:
# 条件を満さないデータから半分抽出する
non_fulfill_cond_data.sample(frac=0.5, random_state=42)

,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend,treatment
12028,10,1) $0 - $100,29.99,1,0,Surburban,1,Phone,Mens E-Mail,0,0,0.0,1
27737,12,1) $0 - $100,29.99,0,1,Urban,0,Phone,Mens E-Mail,0,0,0.0,1
46714,8,3) $200 - $350,202.80,0,1,Surburban,1,Phone,Mens E-Mail,0,0,0.0,1
20422,10,2) $100 - $200,134.31,0,1,Surburban,0,Phone,Mens E-Mail,0,0,0.0,1
13577,11,1) $0 - $100,84.54,0,1,Urban,1,Web,Mens E-Mail,0,0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25243,7,2) $100 - $200,159.08,0,1,Surburban,1,Phone,Mens E-Mail,0,0,0.0,1
11043,10,1) $0 - $100,35.77,0,1,Surburban,0,Phone,Mens E-Mail,0,0,0.0,1
30964,8,1) $0 - $100,67.77,0,1,Urban,0,Phone,Mens E-Mail,0,0,0.0,1
35606,6,1) $0 - $100,95.77,0,1,Surburban,0,Phone,Mens E-Mail,0,0,0.0,1


In [25]:
# 条件を満たさないデータの半分を削除する
biased_data.drop(non_fulfill_cond_data.sample(frac=0.5, random_state=42).index, inplace=True)

In [26]:
biased_data.shape

(31925, 13)

### 集計

In [37]:
# メールが配信されたグループとされなかったグループでの購入の発生確率(conversion)と購入額(spend)の平均を計算
display(biased_data.groupby("treatment").mean()[["conversion", "spend"]])

# メールが配信されたグループとされなかったグループのカウント数
# 書籍と微妙に数値が違うが理由はわからない
print(biased_data["treatment"].value_counts(sort=False))

print("\n比較 バイアスなしデータ")
display(male_data.groupby("treatment").mean()[["conversion", "spend"]])
print(male_data["treatment"].value_counts(sort=False))

,conversion,spend
treatment,,
0,0.004947,0.639756
1,0.013920,1.591952


0    14756
1    17169
Name: treatment, dtype: int64

比較 バイアスなしデータ


,conversion,spend
treatment,,
0,0.005726,0.652789
1,0.012531,1.422617


0    21306
1    21307
Name: treatment, dtype: int64


In [28]:
# 男性向けメールが配信されたグループの購買データ
bias_mens_mail = biased_data[biased_data["treatment"] == 1]["spend"]

# メールが配信されなかったグループの購買データ
bias_no_mail = biased_data[biased_data["treatment"] == 0]["spend"]

In [40]:
print("バイアスあり")
# 2群間でのt検定 書籍と違い、バイアスのないデータよりも p値は 1.27e-07 と若干高い
display(stats.ttest_ind(bias_mens_mail, bias_no_mail, equal_var=True))

# 2群間でのt検定 p-valueは1.163e-07 と小さいので有意
print("\n(再掲)バイアスなし")
display(stats.ttest_ind(mens_mail, no_mail, equal_var=True))

バイアスあり


Ttest_indResult(statistic=5.283787889513689, pvalue=1.273689559022462e-07)


(再掲)バイアスなし


Ttest_indResult(statistic=5.300090294465472, pvalue=1.163200872605869e-07)

### データのCSV出力

In [30]:
male_data.to_csv('male_data.csv', index=False)

biased_data.to_csv('biased_data.csv', index=False)